# 벡터스토어 생성 

In [43]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import CSVLoader

# 텍스트 로더 추가
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain import hub

from langchain_teddynote import logging
from dotenv import load_dotenv
import os

from langchain_google_genai import ChatGoogleGenerativeAI


In [2]:
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
# langsmith 추적 설정
logging.langsmith("bigcon_langchain_test")

LangSmith 추적을 시작합니다.
[프로젝트명]
bigcon_langchain_test


<br><hr>

### 00. 각 지역별 텍스트 파일 로드

In [62]:
# # 각 지역별 텍스트 파일 로드
# file_paths = {
#     # '동부': 'data/data_to_text_by_area/동부.txt',
#     # '서부': 'data/data_to_text_by_area/서부.txt',
#     # '남부': 'data/data_to_text_by_area/남부.txt',
#     # '북부': 'data/data_to_text_by_area/북부.txt',
#     '가파도': 'data/data_to_text_by_area/가파도.txt',
#     '우도': 'data/data_to_text_by_area/우도.txt',
#     '마라도': 'data/data_to_text_by_area/마라도.txt',
#     '비양도': 'data/data_to_text_by_area/비양도.txt',
#     '추자도': 'data/data_to_text_by_area/추자도.txt',
#     'meta_text': 'data/data_to_text_by_area/meta_text.txt'
# }

# loader = TextLoader('..\..\data\data_to_text_by_area\가파도.txt', encoding='utf-8')
# docs = loader.load()
# print(f"문서의 수: {len(docs)}")
# print('[메타데이터]\n')
# print(docs[0].metadata)
# print("\n=========================== [앞부분] 미리보기 ============================\n")
# print(docs[0].page_content[:500])

In [59]:
# 사용자 지역 선택 옵션 추가
visit_region = ['북부', '가파도']

In [14]:
loader = CSVLoader('..\..\data\sample_without_meta.csv', encoding='cp949')
docs = loader.load()
print(f"문서의 수: {len(docs)}")
print('[메타데이터]\n')
print(docs[0].metadata)
print("\n=========================== [앞부분] 미리보기 ============================\n")
print(docs[0].page_content[:500])

문서의 수: 1000
[메타데이터]

{'source': '..\\..\\data\\sample_without_meta.csv', 'row': 0}

=========================== [앞부분] 미리보기 ============================

기준년월: 10월
가맹점명: 가파도용궁정식
업종: 가정식
5시_11시_평균기온: 21.70645161
12시_13시_평균기온: 21.53467742
14시_17시_평균기온: 19.74129032
18시_22시_평균기온: 19.09784946
23시_4시_평균기온: 19.57419355
월_평균기온: 20.33089247
지역: 가파도
주소: 제주 서귀포시 대정읍 가파리 559-1번지 1층
월별_업종별_이용건수_순위: 5
월별_업종별_이용금액_순위: 4
건당_평균_이용금액_순위: 3
월요일_이용_건수_비중: 0.3333333333
화요일_이용_건수_비중: 0.1111111111
수요일_이용_건수_비중: 0.08333333333
목요일_이용_건수_비중: 0.08333333333
금요일_이용_건수_비중: 0.05555555556
토요일_이용_건수_비중: 0.1944444444
일요일_이용_건수_비중: 0.1388888889
5시_11시_이용_건수_비중: 0.1944444444
12시_13


In [46]:
# pandas로 데이터프레임 칼럼명 가져오기
import pandas as pd

csv_path = '..\..\data\sample_without_meta.csv'
df = pd.read_csv(csv_path, encoding='cp949')
column_names = df.columns

In [48]:
# test #
from langchain.schema import Document 

docs = []
for _, row in df.iterrows():
  # 필요한 메타데이터 설정
  metadata = {
    '지역': row['지역'],
    '업종': row['업종']
  }
  # 각 행의 데이터를 문서로 변환
  doc = Document(
    page_content=str(row.to_dict()),
    metadata=metadata
  )
  docs.append(doc)

print(f"문서의 수: {len(docs)}")
print('[메타데이터 예시]\n', docs[0].metadata)
print("\n=========================== [앞부분] 미리보기 ============================\n")
print(docs[0].page_content[:500])

문서의 수: 1000
[메타데이터 예시]
 {'지역': '가파도', '업종': '가정식'}

=========================== [앞부분] 미리보기 ============================

{'기준년월': '10월', '가맹점명': '가파도용궁정식', '업종': '가정식', '5시_11시_평균기온': 21.70645161, '12시_13시_평균기온': 21.53467742, '14시_17시_평균기온': 19.74129032, '18시_22시_평균기온': 19.09784946, '23시_4시_평균기온': 19.57419355, '월_평균기온': 20.33089247, '지역': '가파도', '주소': '제주 서귀포시 대정읍 가파리 559-1번지 1층', '월별_업종별_이용건수_순위': 5, '월별_업종별_이용금액_순위': 4, '건당_평균_이용금액_순위': 3, '월요일_이용_건수_비중': 0.3333333333, '화요일_이용_건수_비중': 0.1111111111, '수요일_이용_건수_비중': 0.08333333333, '목요일_이용_건수_비중': 0.08333333333, '금요일_이용_건수_비중': 0.05555555556, '토요일_이용_건수_비중': 0.1944


In [30]:
# 메타데이터 추가
for idx, doc in enumerate(docs):
  row_data = df.iloc[idx]          # 각 문서의 index를 사용하여 행 데이터 가져오기
  metadata = {
    '지역': row_data['지역'],
    '업종': row_data['업종']
  }
  doc.metadata.update(metadata)

print(f"문서의 수: {len(docs)}")
print('[메타데이터 예시]\n', docs[0].metadata)

문서의 수: 1000
[메타데이터 예시]
 {'source': '..\\..\\data\\sample_without_meta.csv', 'row': 0, '지역': '가파도', '가맹점명': '가정식', '업종': '가정식'}


<br><hr>

### 01. 데이터 청크 나누기 

In [87]:
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000, chunk_overlap=0
)

splits = text_splitter.split_documents(docs)
print("split된 문서의 수:", len(splits))
splits[100:105]

split된 문서의 수: 1000


[Document(metadata={'지역': '남부', '업종': '민속주점'}, page_content="{'기준년월': '9월', '가맹점명': '엉또민속주점', '업종': '민속주점', '5시_11시_평균기온': 25.52333333, '12시_13시_평균기온': 27.78666667, '14시_17시_평균기온': 27.45166667, '18시_22시_평균기온': 25.574, '23시_4시_평균기온': 24.60111111, '월_평균기온': 26.18735556, '지역': '남부', '주소': '제주 서귀포시 강정동 178-1번지', '월별_업종별_이용건수_순위': 4, '월별_업종별_이용금액_순위': 4, '건당_평균_이용금액_순위': 3, '월요일_이용_건수_비중': 0.1063829787, '화요일_이용_건수_비중': 0.2127659574, '수요일_이용_건수_비중': 0.1276595745, '목요일_이용_건수_비중': 0.1276595745, '금요일_이용_건수_비중': 0.1914893617, '토요일_이용_건수_비중': 0.2340425532, '일요일_이용_건수_비중': 0.0, '5시_11시_이용_건수_비중': 0.0, '12시_13시_이용_건수_비중': 0.0, '14시_17시_이용_건수_비중': 0.04255319149, '18시_22시_이용_건수_비중': 0.7872340426, '23시_4시_이용_건수_비중': 0.170212766, '현지인_이용_건수_비중': 0.6727642276, '남성_회원수_비중': 0.611, '여성_회원수_비중': 0.389, '20대_이하_회원수_비중': 0.234, '30대_회원수_비중': 0.285, '40대_회원수_비중': 0.283, '50대_회원수_비중': 0.168, '60대_이상_회원수_비중': 0.031}"),
 Document(metadata={'지역': '남부', '업종': '베이커리'}, page_content="{'기준년월': '1월', '가맹점명': '채점석 베이커리

In [88]:
print("메타데이터 확인:", splits[0].metadata)

메타데이터 확인: {'지역': '가파도', '업종': '가정식'}


<br><hr>

### 02. 벡터스토어 생성

In [133]:
vectorstore = FAISS.from_documents(
  documents=splits, embedding=HuggingFaceEmbeddings(model_name='upskyy/bge-m3-Korean')
)

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/664 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

: 

<br><hr>

### 03. '지역' 필터링

In [129]:
# 사용자 선택 지역과 일치하는 데이터만 가져오기 위한 필터 함수
# def region_filter(doc):
#   region = doc.metadata.get('지역').strip()
#   return region in visit_region

# 검색기 생성
retriever = vectorstore.as_retriever(
  search_type='mmr',
  search_kwargs={"k": 10,              # 반환할 문서 수 (default: 4)
                  "fetch_k": 10,       # MMR 알고리즘에 전달할 문서 수
                  "lambda_mult": 0.5}, # 결과 다양성 조절 (default: 0.5)
  # filter=region_filter
  )

<br><hr>

### 04. 프롬프트 생성

In [130]:
prompt = hub.pull("rlm/rag-prompt")
prompt

ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"), additional_kwargs={})])

In [131]:
system_instruction = (
"""
당신은 제주도 여행객에게 제주도 맛집을 추천하는 '친절한 제주°C' 챗봇입니다. 제공된 데이터를 제시하며 사용자에게 맛집을 추천해주세요"
"""
)

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash",
                             temperature=0,
                             max_tokens=5000,
                             system_instruction=system_instruction
)

# 검색한 문서에서 visit_region에 없는 지역 데이터 필터링
def filter_results_by_region(docs, visit_region):
    return [doc for doc in docs if doc.metadata.get('지역') in visit_region]


# 검색한 문서를 하나의 문단으로 합치기
def format_docs(docs):
  return "\n\n".join(doc.page_content for doc in docs)

# 랭체인 생성
rag_chain = (
  {"context": retriever
   | (lambda docs: filter_results_by_region(docs, visit_region))
   | format_docs,
    "question":RunnablePassthrough()
  }
  # question(사용자의 질문) 기반으로 연관성이 높은 문서 retriever 수행 >> format_docs로 문서를 하나로 만듦
  | prompt               # 하나로 만든 문서를 prompt에 넘겨주고
  | llm                  # llm이 원하는 답변을 만듦
  | StrOutputParser()
)

<br><hr>

### 05. invoke

In [132]:
rag_chain.invoke(
  '3월에 가파도에 갈거야. 3월에 20대에게 인기가 많은 가파도 맛집을 추천해줘'
)

'가파도 3월 20대에게 인기있는 맛집은 가파도해물짜장짬뽕입니다. 이곳은 3월에 중식 업종에서 이용 건수 2위, 이용 금액 1위를 기록했습니다. 20대 이하 회원 비중은 9.7%로 젊은 층에게 인기가 많습니다. \n'

In [95]:
rag_chain.invoke(
  '마라도 맛집 중, 4월 데이터가 있는 맛집의 현지인 이용 건수 비중을 알려줘'
)

'마라도 맛집 중 4월 데이터가 있는 맛집은 마라도해녀촌입니다. 마라도해녀촌의 4월 현지인 이용 건수 비중은 4.79%입니다. \n'

In [38]:
rag_chain.invoke(
  '북부 중식 맛집 중 20대에게 인기 있는 곳을 알려줘'
)

'20대 회원 비중이 높은 북부 중식 맛집은 해신짬뽕과 배셰프입니다. 해신짬뽕은 20대 이하 회원 비중이 12.7%이고, 배셰프는 7.6%입니다. 두 곳 모두 북부 지역에 위치하고 있으며, 중식을 전문으로 합니다. \n'

In [58]:
df[df['지역']=='가파도']['가맹점명'].unique()

array(['가파도용궁정식', '전망대', '가파도바다별장', '가파도핫도그', '블랑로쉐', '가파도해물짜장짬뽕',
       '오멍가멍쉬멍'], dtype=object)